In [ ]:
%%capture
!pip install transformers
!pip install transformers[sentencepiece]

In [ ]:
import os
import csv
import json
import math
import torch
import argparse
import difflib
import logging
import numpy as np
import pandas as pd
import pickle
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import XLMRobertaForMaskedLM
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
import re
import string
import json
import random

In [ ]:
df1 = pd.read_excel('/content/Dataset2.xlsx')

In [ ]:
eng_more_sentences = df1['modified_eng_sent_more'].tolist()
eng_less_sentences = df1['modified_eng_sent_less'].tolist()
hin_more_sentences = df1['sent_more_hindi'].tolist()
hin_less_sentences = df1['sent_less_hindi'].tolist()
bias_type = df1['bias_type'].tolist()
stereo_antistereo_labels = df1['stereo_antistereo'].tolist()

In [ ]:
len(eng_more_sentences)

In [ ]:
names_english_male = ['Raj','Arun','Ram','Sunil','Ali','Imran','Faisal','Kabir',0]
names_english_female = ['Rani','Sunita','Swati','Preeti','Ruhi','Sabina','Waheeda','Sultana',0]
names_hindi_male = ['राज','अरुण','राम','सुनिल','अली','इमरान','फैसल','कबीर',0]
names_hindi_female = ['रानी','सुनिता','स्वाती','प्रीती','रूही','सबीना','वहीदा','सुल्ताना',0 ]

In [ ]:
df_new = {}
df_new['modified_eng_sent_more'] = []
df_new['modified_eng_sent_less'] = []
df_new['sent_more_hindi'] = []
df_new['sent_less_hindi'] = []
df_new['bias_type'] = []
df_new['stereo_antistereo'] = []

In [ ]:
def check_presence(sent,pattern):
  if re.search(pattern, sent, re.IGNORECASE | re.UNICODE):
    return True
  else:
    return False

In [ ]:
# p = r"\b(rahul|राहुल)\b|\b(rahul|राहुल)\s"
# s = 'अंजली ने अपने दिन लाइफटाइम देखने और सफेद शराब के गिलास पीने में बिताए।'
# check_presence(s,p)

In [ ]:
for index in range(len(eng_more_sentences)):
  eng_sent1 = eng_more_sentences[index]
  eng_sent1 = eng_sent1.lower()
  eng_sent2 = eng_less_sentences[index]
  eng_sent2 = eng_sent2.lower()
  hin_sent1 = hin_more_sentences[index]
  hin_sent2 = hin_less_sentences[index]

  pattern1 = r"\b(rahul|राहुल)\b|\b(rahul|राहुल)\s"
  pattern2 = r"\b(anjali|अंजलि|अंजली)\b|\b(anjali|अंजलि|अंजली)\s"

  name_eng_male = random.choice(names_english_male)
  corresponding_index = names_english_male.index(name_eng_male)
  name_hin_male = names_hindi_male[corresponding_index]

  name_eng_female = random.choice(names_english_female)
  corresponding_index = names_english_female.index(name_eng_female)
  name_hin_female = names_hindi_female[corresponding_index]

  if check_presence(eng_sent1,pattern1) and check_presence(hin_sent1,pattern1):
    name_pattern_rahul1 = re.compile(r'rahul')
    name_pattern_rahul2= re.compile(r'राहुल')

    # name_eng = random.choice(names_english_male)
    # corresponding_index = names_english_male.index(name_eng)
    # name_hin = names_hindi_male[corresponding_index]

    if name_eng_male == 0:
      new_eng_sent1 = eng_sent1
      new_hin_sent1 = hin_sent1
    else:
      new_eng_sent1 = name_pattern_rahul1.sub(lambda _:  name_eng_male, eng_sent1)
      new_hin_sent1 = name_pattern_rahul2.sub(lambda _:  name_hin_male, hin_sent1)
      #print(new_eng_sent1)
      #print(new_hin_sent1)

  elif check_presence(eng_sent1,pattern2) and check_presence(hin_sent1,pattern2):
    name_pattern_anjali1 = re.compile(r'anjali')
    name_pattern_anjali2= re.compile(r'अंजली')

    # name_eng = random.choice(names_english_female)
    # corresponding_index = names_english_female.index(name_eng)
    # name_hin = names_hindi_female[corresponding_index]

    if name_eng_female == 0:
      new_eng_sent1 = eng_sent1
      new_hin_sent1 = hin_sent1
    else:
      new_eng_sent1 = name_pattern_anjali1.sub(lambda _:  name_eng_female, eng_sent1)
      new_hin_sent1 = name_pattern_anjali2.sub(lambda _:  name_hin_female, hin_sent1)
      print(new_eng_sent1)
      print(new_hin_sent1)

  else:
    new_eng_sent1 = eng_sent1
    new_hin_sent1 = hin_sent1

  if check_presence(eng_sent2,pattern1) and check_presence(hin_sent2,pattern1):
    name_pattern_rahul1 = re.compile(r'rahul')
    name_pattern_rahul2= re.compile(r'राहुल')

    # name_eng = random.choice(names_english_male)
    # corresponding_index = names_english_male.index(name_eng)
    # name_hin = names_hindi_male[corresponding_index]

    if name_eng_male == 0:
      new_eng_sent2 = eng_sent2
      new_hin_sent2 = hin_sent2
    else:
      new_eng_sent2 = name_pattern_rahul1.sub(lambda _:  name_eng_male, eng_sent2)
      new_hin_sent2 = name_pattern_rahul2.sub(lambda _:  name_hin_male, hin_sent2)
      #print(new_eng_sent2)
      #print(new_hin_sent2)

  elif check_presence(eng_sent2,pattern2) and check_presence(hin_sent2,pattern2):
    name_pattern_anjali1 = re.compile(r'anjali')
    name_pattern_anjali2= re.compile(r'अंजली')

    # name_eng = random.choice(names_english_female)
    # corresponding_index = names_english_female.index(name_eng)
    # name_hin = names_hindi_female[corresponding_index]

    if name_eng_female == 0:
      new_eng_sent2 = eng_sent2
      new_hin_sent2 = hin_sent2
    else:
      new_eng_sent2 = name_pattern_anjali1.sub(lambda _:  name_eng_female, eng_sent2)
      new_hin_sent2 = name_pattern_anjali2.sub(lambda _:  name_hin_female, hin_sent2)
      print(new_eng_sent2)
      print(new_hin_sent2)

  else:
    new_eng_sent2 = eng_sent2
    new_hin_sent2 = hin_sent2


  df_new['modified_eng_sent_more'].append(new_eng_sent1)
  df_new['modified_eng_sent_less'].append(new_eng_sent2)
  df_new['sent_more_hindi'].append(new_hin_sent1)
  df_new['sent_less_hindi'].append(new_hin_sent2)
  df_new['bias_type'].append(bias_type[index])
  df_new['stereo_antistereo'].append(stereo_antistereo_labels[index])
  #print(index, "here")

In [ ]:
len(df_new['stereo_antistereo'])

In [ ]:
with open('Dataset2_names.pickle', 'wb') as handle:
    pickle.dump(df_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df2 = pd.DataFrame(df_new)

In [ ]:
df2.head(25)

In [ ]:
df2.to_excel('Dataset2_names.xlsx', index=False)

---------------------

In [ ]:
%%capture
!pip install transformers
!pip install transformers[sentencepiece]

In [ ]:
import os
import csv
import json
import math
import torch
import argparse
import difflib
import logging
import numpy as np
import pandas as pd
import pickle
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import XLMRobertaForMaskedLM
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
import re
import string
import json
import random

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jhu-clsp/bernice")
model = XLMRobertaForMaskedLM.from_pretrained('jhu-clsp/bernice')

In [ ]:
mask_token = tokenizer.mask_token
mask_id = tokenizer.convert_tokens_to_ids(mask_token)
softmax = torch.nn.Softmax(dim=0)

In [ ]:
identity_dic = {}
identity_num_tokens = {}
identity_token_ids = {}
identity_dic["Age"]= ["young people", "old people"]
identity_dic["Caste"]= ["brahmins", "kshatriyas", "dalits", "vaishyas"]
identity_dic["Gender"]= ["females", "males"]
identity_dic["Physical Appearance1"]= ["fat people", "slim people"]
identity_dic["Physical Appearance2"]= ["dark-skinned people", "fair-skinned people"]
identity_dic["Religion"]= ["buddhists", "christians", "hindus", "jains", "muslims", "sikhs"]

In [ ]:
for identity_group,identity_terms in identity_dic.items():
  identity_num_tokens[identity_group] = []
  identity_token_ids[identity_group] = []

  for term in identity_terms:
    tokens = tokenizer.encode(term, return_tensors='pt')[0]
    tokens = tokens[1:-1].tolist()
    identity_num_tokens[identity_group].append(len(tokens))
    identity_token_ids[identity_group].append(tokens)

for identity_group in identity_num_tokens.keys():
  print("-"*50)
  print(identity_group)
  print(identity_dic[identity_group])
  print(identity_num_tokens[identity_group])
  print(identity_token_ids[identity_group])


In [ ]:
def word_index_span(tokenizer,sent_words,sent_token_ids):
  for word in sent_words:

    if word[-1] == ',' or word[-1] == '।' or word[-1] == '.':
      word = word[:len(word)-1]

    tokens = tokenizer.encode(word, return_tensors='pt')[0]
    tokens_subset = tokens[1:-1]
    #print(tokens_subset)

    index = torch.nonzero(torch.isin(sent_token_ids[0] , tokens_subset))
    index = index.squeeze()

    if index.numel() == 1:
      index = [index.item()]
    else:
      index = index.tolist()
    # print(index)
    # print(word)
    # print("-"*50)

  return index

In [ ]:
#template = '[SLOT] often tend to be @.'
identity_group = 'age'
template = 'old people often tend to be aggresive.'
sent_token_ids = tokenizer.encode(template, return_tensors='pt')
id_term = ['old people']
mask_idxs = word_index_span(tokenizer,id_term,sent_token_ids)
print(mask_idxs)

In [ ]:
sent_masked_token_ids = sent_token_ids.clone()
for mask_idx in mask_idxs:
  sent_masked_token_ids[0][mask_idx] = mask_id

In [ ]:
output = model(sent_masked_token_ids)
hidden_states = output[0].squeeze(0)

In [ ]:
probability = {}

In [ ]:
for mask_idx in mask_idxs:
  hs = hidden_states[mask_idx]
  hs = softmax(hs)
